In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd


In [16]:
mnist = pd.read_csv('Data/iris.csv')
mnist = mnist.sample(frac=1).reset_index().drop('index',axis=1)
mnist.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.9,3.2,4.8,1.8,1.0
1,5.7,2.6,3.5,1.0,1.0
2,7.2,3.2,6.0,1.8,2.0
3,6.3,3.4,5.6,2.4,2.0
4,5.7,2.5,5.0,2.0,2.0


In [17]:
mnist['target'].unique()

array([1., 2., 0.])

In [18]:
train = mnist[:int(len(mnist)*0.8)]
test = mnist[int(len(mnist)*0.8):]
print(f"Shape of entire dataset: {mnist.shape}")
print(f"Shape of train set: {train.shape}")
print(f"Shape of test set: {test.shape}")

Shape of entire dataset: (150, 5)
Shape of train set: (120, 5)
Shape of test set: (30, 5)


In [22]:
train_x = train.drop('target',axis=1)
train_y = train.loc[:,'target']

train_x = torch.FloatTensor(train_x.values)
train_y = torch.LongTensor(train_y.values)

In [23]:
train_x[:10]

tensor([[5.9000, 3.2000, 4.8000, 1.8000],
        [5.7000, 2.6000, 3.5000, 1.0000],
        [7.2000, 3.2000, 6.0000, 1.8000],
        [6.3000, 3.4000, 5.6000, 2.4000],
        [5.7000, 2.5000, 5.0000, 2.0000],
        [6.0000, 2.9000, 4.5000, 1.5000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.8000, 2.7000, 5.1000, 1.9000],
        [5.0000, 3.0000, 1.6000, 0.2000]])

In [24]:
train_y[:10]

tensor([1, 1, 2, 2, 2, 1, 1, 0, 2, 0])

In [25]:
class Model(nn.Module):
    
    def __init__(self,h1,h2,in_features=4,out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2, out_features)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        

In [27]:
epochs = 200
model = Model(5,6)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

for i in range(epochs):
    pred = model.forward(train_x)
    loss = criterion(pred,train_y)
    
    if i%20 == 0:
        print(f"Epoch {i}: Loss is {loss}")
        
    optimizer.zero_grad()     # Resets accumulated gradient
    loss.backward()           
    optimizer.step()          # Backpropagation based on loss.backward() values

Epoch 0: Loss is 1.0416673421859741
Epoch 20: Loss is 0.5783542394638062
Epoch 40: Loss is 0.40150317549705505
Epoch 60: Loss is 0.35533255338668823
Epoch 80: Loss is 0.2985168993473053
Epoch 100: Loss is 0.23450838029384613
Epoch 120: Loss is 0.18378493189811707
Epoch 140: Loss is 0.14520977437496185
Epoch 160: Loss is 0.11906984448432922
Epoch 180: Loss is 0.10243574529886246


In [28]:
pred

tensor([[ -2.8258,   1.6205,   1.6531],
        [ -0.5243,   1.8525,  -1.4626],
        [ -4.5042,   1.4513,   3.9254],
        [ -7.1072,   1.1889,   7.4495],
        [ -6.6832,   1.2316,   6.8754],
        [ -1.0787,   1.7966,  -0.7122],
        [ -0.5243,   1.8525,  -1.4626],
        [  5.1254,  -4.6576,  -5.4466],
        [ -5.7858,   1.3221,   5.6605],
        [  5.1796,  -4.7178,  -5.4853],
        [ -0.5243,   1.8525,  -1.4626],
        [ -0.5243,   1.8525,  -1.4626],
        [  5.2215,  -4.7789,  -5.5117],
        [  6.9042,  -6.7667,  -6.6858],
        [ -0.5243,   1.8525,  -1.4626],
        [ -7.3127,   1.1681,   7.7277],
        [  5.4810,  -5.0797,  -5.6941],
        [ -0.5243,   1.8525,  -1.4626],
        [ -0.5243,   1.8525,  -1.4626],
        [ -7.1943,   1.1801,   7.5674],
        [  5.6611,  -5.2887,  -5.8209],
        [  5.3421,  -4.9099,  -5.5987],
        [ -3.4631,   1.5562,   2.5159],
        [  7.0122,  -6.8885,  -6.7626],
        [ -0.8847,   1.8161,  -0.9747],


In [29]:
test_x = test.drop('target',axis=1)
test_y = test.loc[:,'target']

test_x = torch.FloatTensor(test_x.values)
test_y = torch.LongTensor(test_y.values)

In [41]:
correct = 0
with torch.no_grad():
    pred = model.forward(test_x)
    print("S/N\t\tPred\t\tActual")
    for i in range(len(pred)):
        print(f"{i}\t\t{torch.argmax(pred[i])}\t\t{test_y[i]}")
        if torch.argmax(pred[i]) == test_y[i]:
            correct += 1
    
    print(f"Number of correct values:{correct}/{len(pred)} ({correct*100/len(pred)}%)")

S/N		Pred		Actual
0		0		0
1		2		2
2		1		1
3		2		2
4		2		2
5		1		1
6		1		1
7		2		2
8		1		1
9		0		0
10		2		2
11		0		0
12		0		0
13		0		0
14		2		2
15		2		2
16		2		2
17		2		2
18		1		1
19		1		1
20		2		2
21		1		1
22		2		2
23		1		1
24		2		2
25		1		1
26		2		2
27		1		1
28		0		0
29		1		1
Number of correct values:30/30 (100.0%)


In [38]:
torch.argmax(pred[1])

tensor(2)